<a href="https://colab.research.google.com/github/carlosvneves/fiap_techchallenge04/blob/main/TechChallenge04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# carregamento de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import ipeadatapy as ip
from os.path import exists

from prophet import Prophet
import pytimetk as tk


# Tech Challenge 04 - Alura Pós-Tech/FIAP
---

Notebook para testes e exploração de ideiais na solução do desafio da fase 04 do curso **Alura Pós-Tech/FIAP**.

## Descrição do problema

Você foi contratado(a) para uma consultoria, e seu trabalho envolve analisar os dados de preço do petróleo brent, que pode ser encontrado no site do ipea. Essa base de dados histórica envolve duas colunas: data e preço (em dólares).

Um grande cliente do segmento pediu para que a consultoria desenvolvesse um dashboard interativo e que gere insights relevantes para tomada de decisão. Além disso, solicitaram que fosse desenvolvido um modelo de Machine Learning para fazer o forecasting do preço do petróleo.

Seu objetivo é:

- Criar um dashboard interativo com ferramentas à sua escolha.

- Seu dashboard deve fazer parte de um storytelling que traga insights relevantes sobre a variação do preço do petróleo, como situações geopolíticas, crises econômicas, demanda global por energia e etc. Isso pode te ajudar com seu modelo. É obrigatório que você traga pelo menos 4 insights neste desafio.

- Criar um modelo de Machine Learning que faça a previsão do preço do petróleo diariamente (lembre-se de time series). Esse modelo deve estar contemplado em seu storytelling e deve conter o código que você trabalhou, analisando as performances do modelo.

- Criar um plano para fazer o deploy em produção do modelo, com as ferramentas que são necessárias.

- Faça um MVP do seu modelo em produção utilizando o Streamlit.

# Carrega os dados

In [19]:
# busca os dados do ipeadata ou gravados em arquivo
def get_ipea_data():
    if not exists("data/petr_brent.csv"):

        # baixa os dados do site do ipea
        # ip_metadata = ip.metadata()
        # busca do código da série de Preço - petróleo bruto - Brent (FOB)
        # ip_metadata[ip_metadata["NAME"].str.contains("Brent") == True]

        # seleção da série com dados a partir de 2001
        petr_brent = ip.timeseries("EIA366_PBRENT366", yearGreaterThan=2001)
        petr_brent.to_csv("data/petr_brent.csv")
    else:
        petr_brent = pd.read_csv("data/petr_brent.csv")

    return petr_brent


petr_brent = get_ipea_data()
petr_brent.head()


In [20]:
# gráfico da série temporal
px.line(
    petr_brent,
    x="DATE",
    y="VALUE (US$)",
)


In [26]:
# converte o campo de data para o formato correto
petr_brent["DATE"] = pd.to_datetime(petr_brent["DATE"])
petr_brent = petr_brent.bfill()
petr_brent.info()


# Análise Exploratória da Série

In [27]:
petr_brent.describe()




Os seguinte dados são importantes para compreensão da série temporal:

- Média:  **US 69.83**
- Mediana: **US 66.78**
- Mínimo: **US 9.12**
- Máximo: **US 143.95**

# Análise preliminar 

- Decomposição
- Identificação de Anomalias

In [28]:
anomalize_df = tk.anomalize(
    data=petr_brent,
    date_column="DATE",
    value_column="VALUE (US$)",
    # period        = 7,
    # iqr_alpha     = 0.05, # using the default
    # clean_alpha   = 0.75, # using the default
    # clean         = "min_max"
)


In [29]:
petr_brent.anomalize(date_column="DATE", value_column="VALUE (US$)")


In [30]:
# Plot seasonal decomposition
tk.plot_anomalies_decomp(
    data=anomalize_df,
    date_column="DATE",
    engine="plotly",
    title="Seasonal Decomposition",
    width=1000,
    height=800,
)


In [31]:
tk.plot_anomalies(
    data=anomalize_df, date_column="DATE", engine="plotly", title="Anomalias Detectadas"
)


In [35]:
anomalies = anomalize_df[anomalize_df["anomaly"] == "Yes"]
anomalies.info()


In [57]:
anomalies["DATE"].dt.year.value_counts().sort_index()


In [59]:
anomalies["DATE"].dt.year.plot(kind="hist", bins=13)


Choques exógenos:
- 2008 a 2010: Crise do Subprime
- março de 2020 a  maio de 2023: pandemia de Covid-19
- fevereiro de 2022: invasão da Ucrânia
- 2014-16 Oil Price Collapse 

# Análise Exploratória da Série

# Modelo de Machine Learning

## Reestruturação da rotina de análise exploratória

In [1]:
import pandas as pd
import ipeadatapy as ip
from os.path import exists


# get data from ipea or reads from file
def get_ipea_data():
    if not exists("data/petr_brent.csv"):

        # baixa os dados do site do ipea
        # ip_metadata = ip.metadata()
        # busca do código da série de Preço - petróleo bruto - Brent (FOB)
        # ip_metadata[ip_metadata["NAME"].str.contains("Brent") == True]

        # seleção da série com dados a partir de 2001
        petr_brent = ip.timeseries("EIA366_PBRENT366", yearGreaterThan=2001)
        petr_brent.to_csv("data/petr_brent.csv")
    else:
        petr_brent = pd.read_csv("data/petr_brent.csv")

    return petr_brent


# load data and fill null values
def load_data():

    # get data from ipea or reads from file
    petr_brent = get_ipea_data()

    # convert RAW DATE to datetime
    petr_brent["DATE"] = pd.to_datetime(petr_brent["DATE"])

    # select only DATE and VALUE (US$)
    petr_brent = petr_brent[["DATE", "VALUE (US$)"]]

    # select the first and last date
    start = petr_brent["DATE"].loc[0].date()
    end = petr_brent["DATE"].iloc[len(petr_brent) - 1].date()
    new_dates = pd.date_range(start=start, end=end, freq="D")

    petr_brent = petr_brent.reindex(new_dates)
    petr_brent = petr_brent.rename_axis("date")

    # fill null values
    petr_brent["VALUE (US$)"] = petr_brent["VALUE (US$)"].interpolate().bfill()

    # rename columns
    petr_brent.columns = ["price"]

    return petr_brent


# calculate descriptive statistics
def descriptive_statistics(petr_brent):

    # calculate null values
    null_sum = petr_brent["price"].isnull().sum()
    not_null_sum = len(petr_brent) - null_sum

    print(null_sum, not_null_sum)

    # calculate descriptive statistics
    df_describe = pd.DataFrame({"price": petr_brent.describe()["price"]}).reset_index()
    df_describe["index"] = [
        "Contagem",
        "Média",
        "Desvio Padrão",
        "Mínimo",
        "Quantil 25%",
        "Mediana",
        "Quantil 75%",
        "Máximo",
    ]

    return df_describe


In [37]:
# get data from ipea or reads from file
petr_brent = get_ipea_data()

# convert RAW DATE to datetime
petr_brent["DATE"] = pd.to_datetime(petr_brent["DATE"])

# select only DATE and VALUE (US$)
petr_brent = petr_brent[["DATE", "VALUE (US$)"]]


In [38]:
petr_brent.set_index("DATE", inplace=True)
petr_brent.head()


,VALUE (US$)
DATE,
2002-01-01,NaN
2002-01-02,20.13
2002-01-03,20.47
2002-01-04,21.20
2002-01-05,NaN


In [39]:
# select the first and last date
start = petr_brent.index[0].date()
end = petr_brent.index[len(petr_brent) - 1].date()
new_dates = pd.date_range(start=start, end=end, freq="D")


In [40]:
petr_brent = petr_brent.reindex(new_dates)

# fill null values
petr_brent["VALUE (US$)"] = petr_brent["VALUE (US$)"].interpolate().bfill()


# rename columns
petr_brent.columns = ["price"]
petr_brent.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8169 entries, 2002-01-01 to 2024-05-13
Freq: D
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   8169 non-null   float64
dtypes: float64(1)
memory usage: 127.6 KB


ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements